# Author: Trevor Dorn-Wallenstein
# 11/15/17
# Let's design survey fields of $h+\chi$ Persei that cover a maximum number of a given set of stars.

In [1]:
import numpy as np, astropy.io.fits as fits, matplotlib.pyplot as plt, re
from astropy.table import Table
from scipy.optimize import minimize
import emcee as mc
from survey_tools import *
%matplotlib inline

In [2]:
member_hdu = fits.open('cluster_members.fits')
member_table = Table(member_hdu[1].data)
OB_table = member_table[member_table['SpT'] <= 20]

/Users/carina/anaconda/lib/python3.5/site-packages/astropy/table/column.py:928: RuntimeWarning: invalid value encountered in less_equal
  return getattr(self.data, oper)(other)


In [3]:
OB_list = [Star(ra,dec) for ra,dec in zip(OB_table['RAJ2000'],OB_table['DEJ2000'])]

### Let's try something different. From a list of stars, create a grid of fields that tiles the entire thing. Then list the fields in order of number of stars they cover. Delete the ones that have zero stars. That's the priority list.

In [4]:
def tile_stars(star_list, field_size, overlap_fraction=0.05, margin_fraction=0.1):
    """
    Creates a survey by tiling a field, then deletes the fields that contain no stars
    
    Parameters
    ----------
    star_list : list
        List of `Star` objects
    field_size : float
        Size of camera in arcminutes
    overlap_fraction : float
        Fraction of the size of the camera to overlap the fields for handy image alignment
    margin_fraction : float
        Fraction of the camera to nudge over so that the first image has a margin
        
    Returns
    -------
    field_list : list
        List of `Field` objects
    """
    
    star_coords = np.array([[star.ra,star.dec] for star in star_list])
    
    #Figure out the furthest northeast point to cover
    max_ra = np.max(star_coords[:,0])
    min_ra = np.min(star_coords[:,0])
    max_dec = np.max(star_coords[:,1])
    min_dec = np.min(star_coords[:,1])
    
    #Add the margins to it
    top_edge = max_dec + (margin_fraction*field_size/60.0)
    #subtract off half the field size
    top_center = top_edge - ((field_size/60.0) / 2.0)
    
    #Same but include spherical geometry, ugh
    left_edge = max_ra + (margin_fraction*field_size/60.0)/np.cos(top_center * np.pi / 180)
    left_center = left_edge - ((field_size/60.0) / 2.0)/np.cos(top_center * np.pi / 180)
    
    ras = [left_center]
    decs = [top_center]
    
    current_ra = left_center
    current_dec = top_center
    
    while current_ra >= (min_ra - (margin_fraction*field_size/60.0)/np.cos(top_center * np.pi / 180)):
        
        current_ra -= (1.0 - overlap_fraction)*field_size/np.cos(top_center * np.pi / 180)/60.0
        ras.append(current_ra)
        
    while current_dec >= (min_dec - (margin_fraction*field_size/60.0)):
        
        current_dec -= (1.0 - overlap_fraction)*field_size/60.0
        decs.append(current_dec)
        
    temp_list = [Field(ra,dec,field_size) for ra in ras for dec in decs]
    field_list = []
    
    for field in temp_list:
        num_stars = field.count_stars_in_field(star_list=star_list)
        if num_stars != 0:
            field_list.append(field)
    
    return field_list

In [5]:
fields = tile_stars(OB_list, 3.0)

In [8]:
low_string = '# Region file format: DS9 version 4.1 \nglobal color=yellow dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \nicrs'
med_string = '# Region file format: DS9 version 4.1 \nglobal color=cyan dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \nicrs'
hi_string = '# Region file format: DS9 version 4.1 \nglobal color=magenta dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \nicrs'

ns = []

for field in fields:
    
    reg_str = field.to_region_string()
    n_in_field = field.count_stars_in_field(OB_list)
    ns.append(n_in_field)
    
ns = np.array(ns)

isort = np.argsort(ns)
field_sort = np.array(fields)[isort]

for field,n in zip(field_sort,ns[isort]):
    
    if n_in_field >= 7:
        
        hi_string += ' \n'

        hi_string += reg_str
        
    elif n_in_field >= 3:
        
        med_string += ' \n'

        med_string += reg_str
    
    else:
        
        low_string += ' \n'

        low_string += reg_str

with open('hi_reg.reg','w') as f:
    f.write(hi_string)
with open('med_reg.reg','w') as f:
    f.write(med_string)
with open('low_reg.reg','w') as f:
    f.write(low_string)